In [34]:
def expand(leaderboard, frequent_aa):
    """Expands each peptide/aminoacid in leaderboard by all 18 aminoacid masses."""
    return [i+(j,) for i in leaderboard for j in frequent_aa] 

In [38]:
x = expand([()], [129, 137, 71, 99, 57, 194])
print x
print expand(x, [129, 137, 71, 99, 57, 194])

[(129,), (137,), (71,), (99,), (57,), (194,)]
[(129, 129), (129, 137), (129, 71), (129, 99), (129, 57), (129, 194), (137, 129), (137, 137), (137, 71), (137, 99), (137, 57), (137, 194), (71, 129), (71, 137), (71, 71), (71, 99), (71, 57), (71, 194), (99, 129), (99, 137), (99, 71), (99, 99), (99, 57), (99, 194), (57, 129), (57, 137), (57, 71), (57, 99), (57, 57), (57, 194), (194, 129), (194, 137), (194, 71), (194, 99), (194, 57), (194, 194)]


In [42]:
def cyclicSpectrum(peptide):
    """Input: (aminoacid masses)
     Output: The cyclic spectrum of Peptide."""
    prefixMass = [0]*((len(peptide)+1))
    for i in range(len(peptide)):
        prefixMass[i+1] = prefixMass[i] + peptide[i]
    peptideMass = prefixMass[len(peptide)]
    cyclic_spectrum = [0]
    for i in range(len(prefixMass)-1):
        for j in range(i+1, len(prefixMass)):
            cyclic_spectrum.append(prefixMass[j] - prefixMass[i])
            if i > 0 and j < (len(prefixMass)-1):
                cyclic_spectrum.append(peptideMass - (prefixMass[j] - prefixMass[i]))
    return sorted(cyclic_spectrum) 

In [41]:
cyclicSpectrum((113,129,128,114))

[0, 113, 114, 128, 129, 227, 242, 242, 257, 355, 356, 370, 371, 484]

In [ ]:
from collections import Counter
def score_peptide(peptide, spectrum):
    """Cyclopeptide Scoring Problem: Compute the score of a cyclic peptide against a spectrum.
     Input: (aa masses) and a collection of integers Spectrum. 
     Output: The score of Peptide against Spectrum, Score(Peptide, Spectrum)."""
    spectrum_peptide = cyclicSpectrum(peptide)
    c1, c2 = Counter(spectrum_peptide), Counter(spectrum)
    return sum([min(n, c2[k]) for k,n in c1.items()])

In [44]:
score_peptide((114,128,129,113), [0, 99, 113, 114, 128, 227, 257, 299, 355, 356, 370, 371, 484])

11

In [45]:
score_peptide((), [0, 99, 113, 114, 128, 227, 257, 299, 355, 356, 370, 371, 484])

1

In [ ]:
def linearSpectrum(peptide):
    """Input: An amino acid string Peptide.
     Output: The linear spectrum of Peptide."""
    prefixMass = [0]*((len(peptide)+1))
    for i in xrange(len(peptide)):
        prefixMass[i+1] = prefixMass[i] + peptide[i]
    #print 'prefixMass', prefixMass
    linear_spectrum = [0]
    for i in xrange(len(prefixMass)-1):
        for j in xrange(i+1, len(prefixMass)):
            linear_spectrum.append(prefixMass[j] - prefixMass[i])
    return sorted(linear_spectrum) 

def score_linear_peptide(peptide, spectrum):
    """Compute the score of a linear peptide with respect to a spectrum.
     Input: An amino acid string Peptide and a collection of integers Spectrum.
     Output: The linear score of Peptide with respect to Spectrum, LinearScore(Peptide, Spectrum)."""
    spectrum_linear_peptide = linearSpectrum(peptide)
    c3, c4 = Counter(spectrum_linear_peptide), Counter(spectrum)
    return sum([min(n, c4[k]) for k,n in c3.items()])


In [46]:
score_linear_peptide((114,128,129,113), [0, 99, 113, 114, 128, 227, 257, 299, 355, 356, 370, 371, 484])

8

In [ ]:
def trim_leaderboard(leaderboard, spectrum, N):
    """Input: A collection of peptides Leaderboard, a collection of integers Spectrum, and an integer N.
     Output: The N highest-scoring linear peptides on Leaderboard with respect to Spectrum."""
    scores =  [[score_linear_peptide(peptide, spectrum), peptide] for peptide in leaderboard]
    sorted_scores = sorted(scores, reverse = True)
    if len(leaderboard) <= N:
        return [i[1] for i in sorted_scores]
    else:
        return [i[1] for i in sorted_scores if i[0] >= sorted_scores[int(N)-1][0]]

In [47]:
trim_leaderboard([(113,71,87,101), (71,113,87,101), (101,113,113,101), (101,128,71,87)], [0, 71, 87, 101, 113, 158, 184, 188, 259, 271, 372], 2)

[(113, 71, 87, 101), (71, 113, 87, 101)]

In [50]:
aa = [57, 71, 87, 97, 99, 101, 103, 113, 114, 115, 128, 129, 131, 137, 147, 156, 163, 186]
def leaderboard_cyclopeptide_sequencing(spectrum, N):
    """ Input: An integer N and a collection of integers Spectrum.
     Output: LeaderPeptide after running LEADERBOARDCYCLOPEPTIDESEQUENCING(Spectrum, N)"""
    leaderboard = expand([()], aa)
    leaderpeptide = ()
    parentmass = max(spectrum)
    while len(leaderboard) > 0:
        leaderboard = expand(leaderboard, aa)
        for peptide in leaderboard[:]:
            mass = sum(peptide)
            if mass == parentmass:
                if score_peptide(peptide, spectrum) > score_peptide(leaderpeptide, spectrum):
                    leaderpeptide = peptide
            elif mass > parentmass:
                leaderboard.remove(peptide)
        leaderboard = trim_leaderboard(leaderboard, spectrum, N)   
    return leaderpeptide

In [51]:
ans = leaderboard_cyclopeptide_sequencing([0, 71, 113, 129, 147, 200, 218, 260, 313, 331, 347, 389, 460], 10)
print ans

(147, 71, 129, 113)


In [1]:
def convolution(spectrum):
    
    """Input: A collection of integers Spectrum.
     Output: The list of elements in the convolution of Spectrum. If an element has
     multiplicity k, it should appear exactly k times; you may return the elements in any order."""
    
    spectrum = sorted(spectrum)
    return [i-j for i in spectrum for j in spectrum if i-j > 0]   

In [52]:
from collections import Counter
def convolution_cyclopeptide_sequencing(M, N, spectrum):
    
    #Get the sorted convolution
    convoluted_spectrum = sorted(convolution(spectrum))
    
    #Select elements from convolution betweeen 57 and 200
    aa = [i for i in convoluted_spectrum if 57<=i<=200]
    
    #Select the M most frequent elements between 57 and 200 in the convolution with ties
    if len(aa) < M:
        frequent_aa = aa
    else:
        c = Counter(aa).most_common()
        frequent_aa = [k for (k,v) in c if v >= c[M-1][1]]
    
    leaderboard = expand([()], frequent_aa)
    leaderpeptide = ()
    parentmass = max(spectrum)
    while len(leaderboard) > 0:
        leaderboard = expand(leaderboard, frequent_aa)
        for peptide in leaderboard[:]:
            mass = sum(peptide)
            if mass == parentmass:
                if score_peptide(peptide, spectrum) > score_peptide(leaderpeptide, spectrum):
                    leaderpeptide = peptide
            elif mass > parentmass:
                leaderboard.remove(peptide)
        leaderboard = trim_leaderboard(leaderboard, spectrum, N)   
    return leaderpeptide

In [55]:
ans = convolution_cyclopeptide_sequencing(20, 60, [57, 57, 71, 99, 129, 137, 170, 186, 194, 208, 228, 265, 285, 299, 307, 323, 356, 364, 394, 422, 493])
'-'.join(str(i) for i in ans)

'99-71-137-57-72-57'

In [64]:
#Read input data
f = open('input/rosalind_ba4i.txt')
spectrum = map(int, f.read().strip().split())

In [65]:
#Call function
output = convolution_cyclopeptide_sequencing(20, 373, spectrum)
#Get output in desired format
'-'.join(str(i) for i in output)

'186-186-156-87-99-137-71-99-129-129-97-113-131-99-115'

In [ ]:
#EXERCISE BREAK: Run ConvolutionCyclopeptideSequencing> on Spectrum25 (reproduced below) with N = 1000 and M = 20. 
#Identify the 86 highest-scoring linear peptides.